In [1]:
import json
import pandas as pd
import requests
from tqdm.notebook import tqdm

In [2]:
json_files = [f'products{i}.json' for i in range(1, 6)]
json_files

['products1.json',
 'products2.json',
 'products3.json',
 'products4.json',
 'products5.json']

In [13]:
df = pd.DataFrame(columns=['item_id', 'shop_id', 'comment', 'rating'])
df

,item_id,shop_id,comment,rating


In [14]:
for json_file in json_files[:2]:
    print(f'Processing: {json_file} ...')
    data = json.load(open(json_file, encoding='UTF-8'))
    products = data['data']['sections'][0]['data']['item']
    for product in tqdm(products):
        item_id = product['itemid']
        shop_id = product['shopid']
        url = f"https://shopee.vn/api/v2/item/get_ratings?itemid={item_id}&limit=50&offset=50&shopid={shop_id}"
        response = requests.get(url=url)
        response_data = response.json()
        ratings = response_data['data']['ratings']
        if ratings is None:
            continue

        for rating in ratings:
            comment = rating['comment']
            rating_star = rating['rating_star']
            df.loc[len(df)] = {
                'item_id': item_id,
                'shop_id': shop_id,
                'comment': comment,
                'rating': rating_star
            }

Processing: products1.json ...


  0%|          | 0/500 [00:00<?, ?it/s]

Processing: products2.json ...


  0%|          | 0/500 [00:00<?, ?it/s]

In [15]:
df

,item_id,shop_id,comment,rating
0,21694828011,405987583,Giao hàng nhanh đóng gói cẩn thận mọi người nê...,5
1,21694828011,405987583,"Sản phẩm đẹp, shop giao đúng hàng, vận chuyển ...",5
2,21694828011,405987583,"Đã nhận đủ số lượng hàng, đóng gói cẩn thận, s...",5
3,21694828011,405987583,Chất lượng sản phẩm:ok\nĐúng với mô tả:ok\n\nC...,4
4,21694828011,405987583,Chất lượng sản phẩm:chua ro\nĐúng với mô tả:đú...,5
...,...,...,...,...
48373,3320845775,8778931,Shop giao hàng đầy đủ đúng tất cả các loại mìn...,5
48374,3320845775,8778931,Nạ ok.ko thấy kích ứng hay gì cả.thường thì đắ...,5
48375,3320845775,8778931,"Vừa đặt & giao trong buổi sáng, đủ số lượng, đ...",5
48376,3320845775,8778931,"Shop giao hàng đủ , sản phẩm chất lượng , dùng...",5


In [16]:
df.to_csv('comments_12_with_rating.csv', index=False)